In [60]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F

import os
from torchvision.utils import save_image

In [61]:
batch_size = 256
learning_rate = 0.01
num_epoch = 10

In [62]:
# Create a directory if not exists
sample_dir = "sample_dir"
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
print(sample_dir)

sample_dir


In [63]:
# Set Hyper parameters and other variables to train the model.
batch_size = 100
test_batch_size = 400
epochs = 10
lr = 0.001
#momentum = 0.005
no_cuda = True
seed = 1
log_interval = 200

use_cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

print("set vars and device done")

set vars and device done


In [64]:
mnist_train = dset.MNIST("../", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("../", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [65]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

In [66]:
print(mnist_train[0])

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [67]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,3) # 입력 채널 수, 출력 채널 수 필터 크기
        self.conv2 = nn.Conv2d(6,16,3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))   # reshape Variable
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features = num_features * s
        return num_features
    
model = Net()

In [68]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)#weight_decay=0.9)
#optimizer = optim.SGD(model.parameters(),lr=lr, momentum=0.9,weight_decay=0.1)

In [91]:
#Define Train function and Test function to validate.
def answer(output,target):
    count = 0
    for i, x in enumerate(output):
        maxi = -float('inf')
        for j, y in enumerate(x):
            if maxi < y:
                maxi = j
        if maxi == target[i]:
            count = count + 1
    return count/i
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        optimizer.zero_grad() # 의미         
        criterion = nn.CrossEntropyLoss()
        loss = (criterion(output,target))#torch.sqrt(criterion(output,target))
        loss.backward()
        optimizer.step()
        #print(output.shape,target.shape)
    if epoch %100 == 0:
        print("epoch : ", epoch)
        print("training loss : ",loss.item())
    return data,target,output

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)            
#             print("target")
#             print(target)
#             print("output")
            print(answer(output))
            criterion = nn.CrossEntropyLoss()
            test_loss += (criterion(output, target)).item()
    test_loss /= len(test_loader.dataset)/(test_batch_size)
    print("test loss : ",test_loss)
    return (data, target ,output)        

In [92]:
# Train and Test the model and save it.
print("training")
for epoch in range(1, epochs+1):
    train(model, device, train_loader, optimizer, epoch)
a,b,c = test(model, device, test_loader)


training


TypeError: answer() missing 1 required positional argument: 'target'